# DSCI 100 Group Project -- Aadi Kanwar

## Introduction

Our group has decided to analyze a dataset containing various categorical and numerical facts about individuals (education, income, and credit score being pertinent to this project), classifying the credit score categorically as either "Low" "Average" or "High". An individual's credit score describes their "creditworthiness", i.e their ability and trustworthiness to repay their debts. Therefore one can reason that a high credit score demonstrates a high "creditworthiness". Credit scores can be numerically represented, (and often are), but for the simplicity of this project, we have decided to classify credit scores between three categorical variables, (low, average, and high). If one wishes to consider these categories numerically, we consider a low credit score to be approximately 300, a high credit score to be around 800, and an average credit score to be around 550 (Investopedia, 2023). Given this information, we formulate the question: "What is a credit score of an individual classified as, when considering their income and education?". The specific data set that will be used to answer this question is titled "Credit Score Classification Dataset" and is provided by Sujith K. Mandala on Kaggle ([here is a direct link](https://www.kaggle.com/datasets/sujithmandala/credit-score-classification-dataset/data)). 

## Preliminary Exploratory Data Analysis 

### Reading the dataset into R from the web

In [1]:
install.packages("janitor")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [11]:
library(tidyverse)
library(janitor)

url1 <- "https://raw.githubusercontent.com/aadikanwar/DSCI100_group_project/main/Credit%20Score%20Classification%20Dataset.csv"  # URL for the raw dataset, taken from the uploaded data set from our GitHub repo
download.file(url1, "data/credit_score_data.csv")  # downloading the file as a data file into the working directory 
data <- read_csv("data/credit_score_data.csv")  #reading the file into the notebook, 
head(data)

Rows: 164 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): Gender, Education, Marital Status, Home Ownership, Credit Score
dbl (3): Age, Income, Number of Children

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Age,Gender,Income,Education,Marital Status,Number of Children,Home Ownership,Credit Score
<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
25,Female,50000,Bachelor's Degree,Single,0,Rented,High
30,Male,100000,Master's Degree,Married,2,Owned,High
35,Female,75000,Doctorate,Married,1,Owned,High
40,Male,125000,High School Diploma,Single,0,Owned,High
45,Female,100000,Bachelor's Degree,Married,3,Owned,High
50,Male,150000,Master's Degree,Married,0,Owned,High


### Cleaning and Wrangling the Data 

In [19]:
clean_cs_data <- data |>
    clean_names()  # this is done to clean the column names and follow the proper tidy convention of column names being lowercase and seperated via underscores 

clean_cs_data <- clean_cs_data |>
    mutate(credit_score = as_factor(credit_score))  #converting the variable we wish to predict into a factor, which will prove to be helpful for analysis

clean_cs_data <- clean_cs_data |>
    select(income, education, credit_score)

head(clean_cs_data) 

income,education,credit_score
<dbl>,<chr>,<fct>
50000,Bachelor's Degree,High
100000,Master's Degree,High
75000,Doctorate,High
125000,High School Diploma,High
100000,Bachelor's Degree,High
150000,Master's Degree,High


The data presented is sensibly clean and wrangled appropriately. Each row is a singular observation, each column is a singular variable, and each cell contains a singular value, and so this data can be considered tidy. The names were cleaned to contain no spaces and be lowercase, and the credit_score column was wrangled such that it is represented as a factor, as that is what we wish to predict/classify. Only pertinent columns are selected (predictors: income and education, and the factor credit_score). 

### Creating a Summary Table using Training Data

In [47]:
library(tidymodels)
cs_split <- initial_split(clean_cs_data, prop = 0.75, strata = credit_score)  #splitting the data into training and testing data
cs_training <- training(cs_split)
cs_testing <- testing(cs_split)

summary_table_1 <- cs_training |>  #displaying the count of each pre-existing class in the training data 
    count(credit_score)

summary_table_1

summary_table_2 <- cs_training |>  #displaying the count of the education predictor variables, also showing the types of educations 
    count(education)

summary_table_2

summary_table_3 <- cs_training |>  #displaying the count of the income predictor variables, showing the different income types and values  
    count(income)

summary_table_3



credit_score,n
<fct>,<int>
High,85
Average,26
Low,11


education,n
<chr>,<int>
Associate's Degree,18
Bachelor's Degree,32
Doctorate,22
High School Diploma,23
Master's Degree,27


income,n
<dbl>,<int>
27500,3
32500,4
37500,4
40000,1
45000,1
47500,3
50000,2
52500,4
55000,2
